**Downloading and Cleaning Meterological Data** <br>
Carolyn Vilter <br>
<br>
Inspiration / Meteorological API approach from Varsha Gopalakrishnan<br>
https://github.com/varsha2509/hyperlocal-aq-prediction/blob/master/Notebooks/Daymet-Data-API-Call.ipynb

In [5]:
# Setup
import daymetpy
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np

In [ ]:
# Basis of eventual met dataset: Vishal's air quality sensor data
# Air quality sensors will be the rows/obs of the final dataset

tf_no2 
tf_pm25 

In [ ]:
# Example use of API for one point in Houston
houston_loc = (-95.3698, 29.7604)
houston1 = daymetpy.daymet_timeseries(lon=houston_loc[0], lat=houston_loc[1], start_year=2017, end_year=2018)
houston1

In [10]:
# FIRST: tf_no2 data
tf_no2["lat_short"] = tf_no2["latitude"].round(2)
tf_no2["lon_short"] = tf_no2["longitude"].round(2)
# Create a unique list of lat, long pairs rounded to 2 decimal places
# These are the points we need meteorological data for 
tf_no2_short = tf_no2[["lat_short", "lon_short"]]
tf_no2_short = drop.duplicates(tf_no2_short)

In [ ]:

# for each lat, lon pair in the list, get 2017-2018 data

# then limit to the time range (sept 2017 to april 2018, inclusive?)

# then compress to one row, all obs - averages

# then add lat lon columns

# then append the new row to the main df

# then merge the new, main met df onto vishal's data


# example
houston_loc = (-95.38, 29.77)
houston2 = daymetpy.daymet_timeseries(lon=houston_loc[0], lat=houston_loc[1], start_year=2017, end_year=2018)